# Set up environment
Let's load the basic modules and read the tables from disk.

In [ ]:
import numpy as np
import astropy.units as u
from astropy.table import Table, QTable
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt

#from sklearn.cluster import KMeans
#from sklearn.preprocessing import StandardScaler
#from sklearn.cluster import DBSCAN

We will now load the two tables we generated in the last session.
- `data1` contains the Gaia query in a rectangle around our region of interest, excluding negative parallaxes.
- `cluster1` is our selection on the previous table to filter sources with parallaxes between 5.0 and 5.7 mas.

⚠️ If you want to use the "official" `data1` and `cluster1` tables instead of the ones created by yourself do:
- open a terminal
- Navigate to the data folder  
`cd pysnacks_1/tutorials/data`
- Delete the files `data1.ecsv` and `cluster1.ecsv`.

In [ ]:
import os
for my_file in ['data1', 'cluster1']:
    if not os.path.isfile(f'../data/{my_file}.ecsv'):
        print(f'Using official {my_file} file')
        os.system(f'unzip ../data/backup/{my_file}.ecsv.zip')
        os.system(f'mv {my_file}.ecsv ../data')
    else:
        print(f'Doing nothing because ../data/{my_file}.ecsv already exists')

In [ ]:
data1 = Table.read('../data/data1.ecsv')
cluster1 = Table.read('../data/cluster1.ecsv')

In [ ]:
def plot_variables(data, col1, col2, ax, **kwargs):
    
    pl = ax.scatter(data[col1], data[col2], **kwargs)

    description1 = data1[col1].description        # Obtain column 1 description
    unit1 = data1[col1].unit                      # Obtain column 1 units
    description2 = data1[col2].description        # Obtain column 2 description
    unit2 = data1[col2].unit                      # Obtain column 2 units

    ax.set_xlabel(f"{description1} [{unit1}]") # Set the axis label in the form "Variable description [units]"
    ax.set_ylabel(f"{description2} [{unit2}]") # Set the axis label in the form "Variable description [units]"

    ax.legend()
    return pl

# Introduction to scikit-learn

[scikit-learn](https://scikit-learn.org/) is a library for Machine Learning in Python. 
We will work with a couple of [clustering algorithms](https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html#sphx-glr-auto-examples-cluster-plot-cluster-comparison-py): [K-means clustering](https://scikit-learn.org/stable/modules/clustering.html#k-means) (you can find an example [here](https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_iris.html#sphx-glr-auto-examples-cluster-plot-cluster-iris-py)) and [DBSCAN](https://scikit-learn.org/stable/modules/clustering.html#dbscan) (you can find an example [here](https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html?highlight=example%20dbscan))   

![image](../data/sphx_glr_plot_cluster_comparison_001.png)

## Data preparation
We need an array of size NxM, with N the number of rows and M the number of features (or columns, or variables). We will forget about the units and work with plain numpy arrays. We will create a numpy array from three of the columns we want. However, note that that array would be of size 3 x 23050, so we need to transpose it with method `.T`.

In [ ]:
pos_tmp = np.array([data1['pmra'],
                    data1['pmdec'], 
                    data1['parallax']
                   ])
pos_tmp.shape   # We cannot use this, we need to transpose the array.

In [ ]:
pos = pos_tmp.T
pos.shape

In [ ]:
pos

## KMeans

In [ ]:
from sklearn.cluster import KMeans

We can learn more about this functions by running:

```python
KMeans?

    KMeans(
        n_clusters=8,
        *,
        init='k-means++',
        n_init=10,
        max_iter=300,
        tol=0.0001,
        verbose=0,
        random_state=None,
        copy_x=True,
        algorithm='auto',
    )
...
```

In [ ]:
KMeans?

[K-means vlustering visualization](https://www.naftaliharris.com/blog/visualizing-k-means-clustering/)

Let's run the algorithm trying to search for three clusters, using `n_clusters=3`.

In [ ]:
kmeans = KMeans(n_clusters=3)    # We initialize the algorithm
kmeans.fit(pos)                  # We conduct the fit
labels = kmeans.predict(pos)     # We extract the labels with the classification
labels

The `labels` can be 0, 1 or 2, and there is one label for each star (in total 23050). Let's see how many stars are in each label:

In [ ]:
plt.hist(labels, bins = np.arange(-0.25, 2.4, 0.5));

In [ ]:
# Finding the centroids
centroids = kmeans.cluster_centers_
centroids

In [ ]:
col1 = 'pmra'
col2 = 'pmdec'

fig, ax = plt.subplots(figsize=(18,10))   # We can select ncols, nrows, or both.

plot_variables(data1[labels==0], col1, col2, ax, s=8, alpha=0.8, label='Group 0')
plot_variables(data1[labels==1], col1, col2, ax, s=8, alpha=0.8, label='Group 1')
plot_variables(data1[labels==2], col1, col2, ax, s=8, alpha=0.8, label='Group 2')

ax.set_aspect('equal')
ax.set_xlim(-60, 30)
ax.set_ylim(-60, 30);

Later we will see why we get the result, for the moment, try to find the cluster using more clusters.

### ⛏ Exercise 4.1
Run the KMeans algorithm again but with selecting 10 clusters `n_clusters=10`.

Tip: when you plot the results, you can use a `for` loop to avoid writing to much.

This is quite dissapoining. This is more useful for cutting a pizza than for finding clusters. The main problems are:
- This is a purely geometric method, so that is what we get: geometric cuts
- The proper motion values are much larger (about 40 mas/yr) and the parallax values (about 5 mas), so the parallax is not contributing

What we really need to do is to normalize the variables so they have comparable value scale.

### Feature scaling

For K-Means Clustering, the Euclidean distance is important, so Feature Scaling makes a huge impact. We can use `StandardScaler` to regularize the data so each feature has mean = 0 and standard deviation = 1.

In [ ]:
from sklearn.preprocessing import StandardScaler
#StandardScaler?

In [ ]:
scaler = StandardScaler()
pos_norm = scaler.fit_transform(pos)
pos_norm    # Now all values are comparable and close to 0

Let's quickly compare the mean and standard deviations before and after the normalization

In [ ]:
print('Before scaling:')
print(f"Mean: {pos.mean(axis=0)}")
print(f"Std:  {pos.std(axis=0)}")
print('\nAfter scaling:')
print(f"Mean: {pos_norm.mean(axis=0)}")
print(f"Std:  {pos_norm.std(axis=0)}")

We are ready to execute again the algorithm but using `pos_norm`

In [ ]:
kmeans = KMeans(n_clusters=3)          # We initialize the algorithm
kmeans.fit(pos_norm)                   # We conduct the fit
labels_norm = kmeans.predict(pos_norm) # We extract the labels with the classification
labels_norm

In [ ]:
col1 = 'pmra'
col2 = 'pmdec'

fig, ax = plt.subplots(figsize=(18,10))   # We can select ncols, nrows, or both.

for label in range(3):
    plot_variables(data1[labels_norm==label], col1, col2, ax, s=8, alpha=0.8, label=f"Group {label}")

ax.set_aspect('equal')
ax.set_xlim(-60, 30)
ax.set_ylim(-60, 30);

Not really what we are looking for. It is purely forcing the groups to follow linear distance, without taking into consideration the density, and that means that many outlier stars are included in the orange group simply because it is the closer group. We need to find an alternative method!

## DBSCAN: Density-Based Spatial Clustering of Aplications with Noise

[DBSCAN visualization](https://www.naftaliharris.com/blog/visualizing-dbscan-clustering/)

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
db = DBSCAN(eps=1.2, min_samples=60, metric='euclidean').fit(pos)
labels_dbscan = db.labels_
labels_dbscan

We can use `set` to remove duplicates so we can see the unique labels

In [ ]:
set(labels_dbscan)

Labeled as -1 for “noise”.

In [ ]:
col1 = 'pmra'
col2 = 'pmdec'

fig, ax = plt.subplots(figsize=(18,10))   # We can select ncols, nrows, or both.

plot_variables(data1[labels_dbscan==label], col1, col2, ax, s=5, c='k', alpha=0.5, label=f"Group {label}")
for label in list(set(labels_dbscan)):
    if label > -1:
        plot_variables(data1[labels_dbscan==label], col1, col2, ax, s=8, alpha=0.8, label=f"Group {label}")

ax.set_aspect('equal')
ax.set_xlim(-60, 30)
ax.set_ylim(-60, 30);

Let's now use `pos_norm` instead of `pos` 

In [ ]:
db_norm = DBSCAN(eps=0.12, min_samples=50, metric='euclidean').fit(pos_norm)
labels_dbscan_norm = db_norm.labels_
labels_dbscan_norm
print(set(labels_dbscan_norm))

col1 = 'pmra'
col2 = 'pmdec'

fig, ax = plt.subplots(figsize=(18,10))   # We can select ncols, nrows, or both.

plot_variables(data1[labels_dbscan_norm==label], col1, col2, ax, s=5, c='k', alpha=0.5, label=f"Group {label}")
for label in list(set(labels_dbscan_norm)):
    if label > -1:
        plot_variables(data1[labels_dbscan_norm==label], col1, col2, ax, s=8, alpha=0.8, label=f"Group {label}")

ax.set_aspect('equal')
ax.set_xlim(-60, 30)
ax.set_ylim(-60, 30);


We can count how many members are in the orange cluster by selecting from `data1` only those stars labelled with label 1.

In [ ]:
len(data1[labels_dbscan_norm==1])

We can show the distribution for the three relevant columns and where the clusters are located

In [ ]:
cols = ['pmra', 'pmdec', 'parallax']
bins = [
    np.arange(-50, 30, 0.5),
    np.arange(-40, 20, 0.5),
    np.arange(0, 7, 0.05)
]

fig, axs = plt.subplots(nrows=3, figsize=(10, 18))

for col, ax, bin in zip(cols, axs, bins):
    ax.hist(data1[col], bins=bin, color='grey', label='Full sample')

    # We iterate now over all the clusters found
    for label in list(set(labels_dbscan_norm)):
        if label > -1:
            ax.hist(data1[labels_dbscan_norm==label][col], bins=bin, label=f'Cluster DBSCAN {label}')

    description = data1[col].description
    unit = data1[col].unit

    ax.set_xlabel(f"{description} [{unit}]")
    ax.legend();

We can also compare the parallax distribution of the `cluster1` we manually selected vs the DBSCAN one

In [ ]:
col1 = 'parallax'
fig, ax = plt.subplots(ncols=2, figsize=(18,6))

# Left panel original cluster1 with a manual cut in parallax
ax[0].hist(data1[col1],    bins=np.arange(4, 7, 0.03), label='Full sample')
ax[0].hist(cluster1[col1], bins=np.arange(4, 7, 0.03), label='Cluster 1')

# Right panel with the cluster found with DBSCAN
ax[1].hist(data1[col1], bins=np.arange(4, 7, 0.03), label='Full sample')
label = 1  # We are focusing now in our cluster
ax[1].hist(data1[labels_dbscan_norm==label][col1], bins=np.arange(4, 7, 0.03), label=f'Cluster DBSCAN {label}')

description = data1[col1].description
unit = data1[col1].unit

ax[0].set_xlabel(f"{description} [{unit}]")
ax[1].set_xlabel(f"{description} [{unit}]")
ax[0].legend()
ax[1].legend();

## Save the new cluster as `cluster2`

We are happy with the cluster selection using DBSCAN, so we will create a new table and save it on disk.

In [ ]:
cluster2 = data1[labels_dbscan_norm==1]
cluster2.write('../data/cluster2.ecsv', format='ascii.ecsv', overwrite=True)

# Can we measure the age of the cluster?

The general method we will use to estimate the age of our cluster is to identify the isochrone tha better fits the turnover between the line of main sequence stars and the red giant branch. A general description of the method is described in [Measuring the Age of a Star Cluster](https://www.e-education.psu.edu/astro801/content/l7_p6.html).

The method is also described in this SEA proceedings: [Rio et al 2014](https://www.sea-astronomia.es/sites/default/files/archivos/proceedings11/via_lactea/rioj/rioj.pdf)

![myimage.gif](../data/HR_age.gif)

Although Gaia does not provide the most accurate photometry, it is equiped with three filters that are good enough for our first analysis. The variables we want to use are:
- `bp_rp`: The BP - RP colour computed from the Blue Passband and the Red Passband 
- `Mg`: The absolute magnitude we computed using the apparent magnitude `phot_g_mean_mag` and the `parallax`

And we can also use a couple other variables to make the plots more interesting:
- `dr2_rv_template_teff`

![image.png](../data/GaiaDR2Passbands.png)

First, let's count how many stars

In [ ]:
print(f"Number of stars in the cluster: {np.count_nonzero(cluster2)}")

print(f"Number of stars with bp_rp values: {np.count_nonzero(cluster2['bp_rp'])}")
print(f"Number of stars with Mg values: {np.count_nonzero(cluster2['bp_rp'])}")
print(f"Number of stars with dr2_rv_template_teff values: {np.count_nonzero(cluster2['dr2_rv_template_teff'])}")


First we will create a HR-diagram for all stars in the original sample

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))

plot_variables(data1, 'bp_rp', 'Mg', ax, c='grey', s=1, label='Full sample')

ax.set_ylabel('Absolute magnitude')   # We didn't write description and units to the original table

ax.set_xlim(-0.5, 3.6)
ax.set_ylim(15, -2.5);

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))

plot_variables(data1,    'bp_rp', 'Mg', ax, c='grey', s=1, label='Full sample')
plot_variables(cluster2, 'bp_rp', 'Mg', ax, c='k', s=50, label='Cluster2')

ax.set_ylabel('Absolute magnitude')   # We didn't write description and units to the original table

ax.set_xlim(-0.5, 3.6)
ax.set_ylim(15, -2.5);

Note: can you identify the binary stellar systems in the plot? And the white dwarfs?

In [ ]:
fig, ax = plt.subplots(figsize=(14,12))

plot_variables(data1,    'bp_rp', 'Mg', ax, c='grey', s=1, label='Full sample')
plot_variables(cluster2, 'bp_rp', 'Mg', ax, c='k', s=50)
pl = plot_variables(cluster2, 'bp_rp', 'Mg', ax, c=cluster2['dr2_rv_template_teff'], s=50, label='Cluster2')

ax.set_ylabel('Absolute magnitude G')   # We didn't write description and units to the original table

cb = fig.colorbar(pl)
cb.set_label("$T_{eff}$ [K]")

ax.set_xlim(-0.5, 3.6)
ax.set_ylim(15, -2.5);

# Isochrones

[Stellar isochrones](https://en.wikipedia.org/wiki/Stellar_isochrone) are curves in the Hertzsprung-Russell computed using stellar evolution models that represent a population of stars of the same age but with different mass.

![image.png](../data/Isochrones_of_several_ages.png)

How to obtain already computed isochrones? It is not straightforward, because many possible curves can be simulated using stellar population synthesis modeling over large grids of stellar parameters like mass, metallicity, size, gravity, etc. Some packages are able to simulate stellar populations based on these templates, for example:
- [SPISEA: Stellar Population Interface for Stellar Evolution and Atmospheres](https://github.com/astropy/SPISEA) PySnacks 2!
- [isochrones](https://isochrones.readthedocs.io/en/latest/) library

Both are based on the MESA Isochrones & Stellar Tracks ([MIST](http://waps.cfa.harvard.edu/MIST/)) database. They provide a very useful web interface to generate isochrones: [Isochrone Interpolation](http://waps.cfa.harvard.edu/MIST/interp_isos.html). We generated a grid of isocrhones for a wide range of ages based on standard stellas paramters.

In particular, we are going to use the file `pysnacks1/data/MIST_iso_62321da9c816b.iso.cmd` that is already uploaded in the repository.

In [ ]:
#New
import os
filename = '../data/MIST_iso_6245f451cfcaf.iso.cmd'

if not os.path.isfile(filename):
    os.system("cd ../data && unzip MIST_iso_6245f451cfcaf.iso.zip")

Great, now we have our isochrones ready, but to load them we need to use a special script to be able to read the cmd format.

The Github repository [MIST_codes](https://github.com/jieunchoi/MIST_codes) provides routines to run and process a grid of MIST models. In particular we want the `read_mist_models.py` script to be able to load MIST models.

In [ ]:
if not os.path.isfile('read_mist_models.py'):
    os.system('curl -O https://raw.githubusercontent.com/jieunchoi/MIST_codes/master/scripts/read_mist_models.py')

In [ ]:
import read_mist_models

In [ ]:
isocmd = read_mist_models.ISOCMD(filename)

In [ ]:
print('version: ', isocmd.version)
print('abundances: ', isocmd.abun)
print('rotation: ', isocmd.rot)
print('ages: ', [round(x,2) for x in isocmd.ages])
print('number of ages: ', isocmd.num_ages)
print('available columns: ', isocmd.hdr_list)

We can access the isochrone for a specific age selecting the age index and the column:

```python
age_ind = isocmd.age_index(age)       # returns the index for the desired age
isocmd.isocmds[age_ind][column_name]
```

For example to obtain the isocrhone corresponding to the Gaia EDR3 G band for an log10(age) of 8.0, que can do:

```python
age_ind = isocmd.age_index(8.0)       # returns the index for the desired age
isocmd.isocmds[age_ind]['Gaia_G_EDR3']
```

It is also possible to select different stellar evolutionary stages by creating a mask. To select stars in the main sequence and red giant phases we can create this mask:
```python
phase_mask = (isocmd.isocmds[age_ind]['phase'] >= 0) & (isocmd.isocmds[age_ind]['phase'] < 3)
isocmd.isocmds[age_ind]['Gaia_G_EDR3'][phase_mask]
```
More details on how to use MIST synthetic profiles can be found [here](https://datacarpentry.org/astronomy-python/calculating_MIST_isochrone/).

In [ ]:
age_ind = isocmd.age_index(8.0)       # returns the index for the desired age
isocmd.isocmds[age_ind]['Gaia_G_EDR3']

As an example, we can plot the isochrone corresponding to log10(age) = 9

In [ ]:
fig, ax = plt.subplots(figsize=(14,12))

plot_variables(data1,    'bp_rp', 'Mg', ax, c='grey', s=1, label='Full sample')
plot_variables(cluster2, 'bp_rp', 'Mg', ax, c='k', s=50)
pl = plot_variables(cluster2, 'bp_rp', 'Mg', ax, c=cluster2['dr2_rv_template_teff'], s=50, label='Cluster2')

ax.set_ylabel('Absolute magnitude G')   # We didn't write description and units to the original table

cb = fig.colorbar(pl)
cb.set_label("$T_{eff}$ [K]")

ax.set_xlim(-0.5, 3.6)
ax.set_ylim(15, -2.5)

age = 9
age_ind = isocmd.age_index(age) #returns the index for the desired age
BP = isocmd.isocmds[age_ind]['Gaia_BP_EDR3']
RP = isocmd.isocmds[age_ind]['Gaia_RP_EDR3']
G  = isocmd.isocmds[age_ind]['Gaia_G_EDR3']
ax.plot(BP-RP, G, label="age", lw=2) 

ax.legend()

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(14,12))

ax.scatter(data1['bp_rp'], data1['Mg'], c='grey', s=1)
ax.scatter(cluster2['bp_rp'], cluster2['Mg'], c='k', s=40)
l = plt.scatter(cluster2['bp_rp'], cluster2['Mg'], c=cluster2['dr2_rv_template_teff'], s=40)

ax.set_ylabel('Absolute magnitude G')   # We didn't write description and units to the original table

cb = fig.colorbar(l)
cb.set_label("$T_{eff}$ [K]")

ax.set_xlim(-0.5, 3.6)
ax.set_ylim(15, -2.5)

for age in [8.7, 8.8, 8.9, 9.0, 9.1]:
    age_ind = isocmd.age_index(age) #returns the index for the desired age
    BP = isocmd.isocmds[age_ind]['Gaia_BP_EDR3']
    RP = isocmd.isocmds[age_ind]['Gaia_RP_EDR3']
    G  = isocmd.isocmds[age_ind]['Gaia_G_EDR3']
    ax.plot(BP-RP, G, label=age, lw=2) 

ax.legend();

### ✨ Exercise 4.2

Estimate the age of the cluster. Although we are not going to fit the isochrone to the data, we can estimage the age of the cluster by selecting the isochrone that better approaches the data, in particular the red giant turnover. Remember that the isochrones are identified by the log10 of the age. Complete this code to find the age of the cluster in Myr.
```python
age_cluster2 = 10**<your preferred isochrone>*u.yr    # Select a number between 8.7 and 9.1
print(f"The age of the cluster is approximately {age_cluster2.to('Myr'):5.3f}") 
```


### ✨ Exercise 4.3:  Summary of final results

- Convert the table `cluster2` to into a QTable with `QTable(cluster2)`
- Print all the relevant values you found for this cluster:
  - Number of members
  - Mean R.A. (in hms) and Dec. (in dms)
  - Mean proper motion in R.A. and Dec. in mas/yr
  - Mean parallax in mas and mean distance in pc.

Tip 1: To make everything easier, work with a Qtable: `c = QTable(cluster2)`  
Tip 2: Define an SkyCoord object using `ra` and `dec` columns and print them with `coord.ra.to_string()`, `coord.dec.to_string()`  
Tip 3: Use prints similar to `print(f"Proper motion R.A. : {c['pmra'].mean():5.1f}")`

### 🌪 Exercise
Write the results as a latex table with two columns: variable name and value.

### 🌪 Exercise
Compare the number of members, proper motions and distance values we obtained with the ones in Vizier table: J/A+A/633/A99/table1

You have two options:  
(a) Download the Vizier table with `astroquery` as explained the first day  
(b) Simply visit https://vizier.cds.unistra.fr/viz-bin/VizieR-5?-ref=VIZ6232587c2323a1&-out.add=.&-source=J/A%2bA/633/A99/table1&recno=834

What is our discrepancy in the distance of the cluster with respect to that publication?

### 🌪 Exercise
Try to improve the proper motion of the cluster by computing the weighted average of the `pmra` and `pmdec` columns. You can do it by converting the column to a numpy array with the `.data` attribute, and then applying `np.average`. Check the documentation.

### 🌪 Exercise
The [Simbad page for NGC 2632 / M44](https://simbad.u-strasbg.fr/simbad/sim-basic?Ident=m44) indicates that:

> Angular size (arcmin): 	118.2 118.2 0 (Opt) D [2020A&A...633A..99C ](https://simbad.cds.unistra.fr/simbad/sim-ref?bibcode=2020A%26A...633A..99C)

Compute the physical size of `cluster2` following the criteria of the paper. Or at least, compute the maximum separation between stars in `cluster2`.

- Looking at that size of the cluster, did we made a good choice when we queried a region of 3 x 2 deg? Could that explain that the number of members we found is smaller than the 697 detected in [this publication](https://vizier.cds.unistra.fr/viz-bin/VizieR-5?-ref=VIZ6232587c2323a1&-out.add=.&-source=J/A%2bA/633/A99/table1&recno=834)?